<a href="https://colab.research.google.com/github/smfs18/projeto-estatistica/blob/BernardoSiqueiraBatista-patch-1/TratamentoDeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tratamento de Dados**
**Esse google colab contém o código feito em R para o tratamento de dados, listando o que foi feito por meio de comentários dos procedimentos realizados **


# Importação de pacotes

Aqui foi feito a importação do pacote tidyverse que será essencial para futuras operações

In [ ]:
install.packages(
  "tidyverse"  # dplyr, tidyr, readr, stringr, etc.
)


library(tidyverse)



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# Planilha de Dados

Pegamos o link do nosso dataset em formato csv e criamos um data_frame df que lê
tal link

In [ ]:
csv_url <- "https://docs.google.com/spreadsheets/d/1t1uBaxiFaByBsNtk9vaLxidQeaNp0P-w40G_sViKvyA/export?format=csv&gid=443957407"

df = read.csv(csv_url)



# Tratamento da coluna "Sexo" do Dataframe

Aqui padronizamos a coluna de maneira que só existam "M": Para Masculino e F para feminino. Além disso, caso exista algum valor vazio. Substituímos por N/A

In [ ]:
#Substituir os valores Masculino por M e os valores Femininos por F.

df$Sexo[df$Sexo == "Masculino"] <- "M"
df$Sexo[df$Sexo == "Feminino"] <- "F"

#Verificar se existem valores vazios, e substitui-los por N/A.

if (any(is.na(df$Sexo) | df$Sexo == "")) {
  df$Sexo[is.na(df$Sexo) | df$Sexo == ""] <- "N/A"
}
head(df$Sexo, 10)

[1] "F" "M" "F" "M" "M" "F" "M" "M" "M" "F"

# Tratamento de Colunas Númericas

Nesse caso, criamos uma função que itera sobre as colunas do nosso DataFrame, caso seja númerica, vamos arredonda em duas casas decimais.

In [ ]:
cols_num <- c("Peso_kg",
              "Altura_cm",
              "Horas_Sono",
              "Pressao_Sistolica",
              "Pressao_Diastolica")

df <- df %>%
  # 1. Converte strings para número, trocando vírgula por ponto se existir
  mutate(across(all_of(cols_num), ~ as.numeric(str_replace_all(.x, ",", ".")))) %>%

  # 2. Arredonda só as colunas que pedem casas decimais
  mutate(
    across(c(Peso_kg, Altura_cm, Horas_Sono),  ~ round(.x, 2)),  # 2 casas
    across(c(Pressao_Sistolica, Pressao_Diastolica), ~ round(.x)) # inteiros
  )


Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `across(...)`.
Caused by warning:
! NAs introduced by coercion”


# Alteração da Coluna Histórico_Doença

Alterando a coluna Histórico_Doença para Sim ou Não.

In [ ]:
df$Historico_Doenca[df$Historico_Doenca == "N"] <- "Não"
df$Historico_Doenca[df$Historico_Doenca == "S"] <- "Sim"

df$Historico_Doenca[df$Historico_Doenca == "nao"] <- "Não"
df$Historico_Doenca[df$Historico_Doenca == "sim"] <- "Sim"



# Tratamento da Coluna de Consumo_Alcool_semanal_ml

Agora, vamos tratar a coluna Consumo_Alcool_semanal_ml, por meio de conversão para númerico e remoção de valores fora da realidade

In [ ]:
df$Consumo_Alcool_semanal_ml <- as.character(df$Consumo_Alcool_semanal_ml)

# Criar nova coluna com os valores limpos
df$Consumo_Alcool_semanal_ml_limpo <- sapply(df$Consumo_Alcool_semanal_ml, function(x) {
  if (is.na(x)) return(NA_real_)

  # Se tiver mais de um ponto, assume que são separadores de milhar (ex: 1.247.546)
  if (str_count(x, "\\.") > 1) {
    x_clean <- str_replace_all(x, "\\.", "")  # remove todos os pontos
  } else {
    x_clean <- x  # mantém ponto decimal válido
  }

  # Tenta converter para numérico
  num <- suppressWarnings(as.numeric(x_clean))

  # Se a conversão falhar ou for absurdamente alta, define como NA
  if (is.na(num) || num > 5000) return(NA_real_)

  return(num)
})

df$Consumo_Alcool_semanal_ml <- df$Consumo_Alcool_semanal_ml_limpo

df$Consumo_Alcool_semanal_ml_limpo <- NULL


# Alteração de Coluna de Data_Exame

Agora, converendo a Data para o Padrão Brasileiro

In [ ]:
df$Data_Exame <- as.Date(df$Data_Exame, format = "%Y-%m-%d")
df$Data_Exame <- format(df$Data_Exame, "%d/%m/%Y")


# Tratamento de Valores N/A

  Agora, vamos substituir todas as colunas numéricas com sua respectivas medianas e os valores NA de tabelas escritas como não informados.Fazemos isso, pela simplicade, eficiência, respeito à distribuição de Dados e por não ser afetada por valores extremos

In [ ]:
library(dplyr)

df <- df %>%
  mutate(across(where(is.numeric), ~replace_na(., median(., na.rm = TRUE))))


df <- df %>%
  mutate(across(where(is.character), ~replace(., . == "NA", "Não informado")))

df <- df %>%
  mutate(across(where(is.character), ~replace_na(., "Não informado")))


,ID_Paciente,Idade,Sexo,Peso_kg,Altura_cm,Fumante,Nivel_Atividade,Horas_Sono,Pressao_Sistolica,Pressao_Diastolica,Historico_Doenca,Consumo_Alcool_semanal_ml,Data_Exame
,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1,45,F,74.07,172.26,Sim,Sedentário,5.1,106,78,Não,134.93,05/04/2024
2,2,55,M,70.72,181.97,Não,Baixo,4.0,135,76,Não,440.33,22/04/2024
3,3,61,F,57.41,171.51,Não,Alto,4.3,112,99,Sim,56.91,21/03/2024
4,4,43,M,75.72,191.34,Não,Moderado,9.2,133,73,Sim,86.24,04/02/2024
5,5,60,M,53.16,171.56,Não,Moderado,6.7,134,79,Não,5.32,25/10/2024
6,6,65,F,88.87,171.91,Sim,Baixo,5.0,110,95,Não Informado,101.58,11/02/2023
7,7,46,M,73.19,147.00,Não,Baixo,7.0,120,90,Não,492.17,24/09/2024
8,8,64,M,69.40,171.13,Não,Moderado,9.6,116,81,Não,6.90,18/06/2024
9,9,33,M,46.34,157.85,Não,Baixo,7.5,111,72,Não,56.34,05/05/2023


# Criando a pasta e o arquivo

In [ ]:
# cria uma subpasta, se não existir
dir.create("/content/saida", showWarnings = FALSE, recursive = TRUE)

# salva o dataframe na nova pasta
caminho <- "/content/saida/dados_tratados.csv"
readr::write_csv(df, caminho)

message("Arquivo salvo em: ", caminho)

Arquivo salvo em: /content/saida/dados_tratados.csv

